In [1]:
import pandas as pd
import numpy as np

In [2]:
try:
    import google.colab
    colab = True
    from google.colab import drive
    drive.mount('/content/drive')
    
    fine_tuned_filepath = '/content/drive/MyDrive/NLP_files/mybert.pickle'
    concepts_filepath = '/content/drive/MyDrive/NLP_files/concepts_strings_with_ids.csv'
    embeddings_output_filepath = '/content/drive/MyDrive/NLP_files/concepts_embeddings.csv'
    device_name = 'cuda'
    import sys
    sys.path.append('/content/drive/MyDrive/NLP/bert_embeddings')
except:
    colab = False
    fine_tuned_filepath = '../models/mybert.pickle'
    concepts_filepath = '../data/concepts_strings_with_ids.csv'
    embeddings_output_filepath = '../results/embeddings/concepts_embeddings.csv'
    device_name = 'cpu'

In [3]:
#!pip install transformers

In [4]:
#!rm -R /content/drive/MyDrive/NLP
#!git clone https://github.com/hadaszm/NLP.git /content/drive/MyDrive/NLP

In [5]:
from transformers import BertTokenizer, BertForPreTraining, BertModel
import torch
from emb_helpers import * 

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_pretrained = BertModel.from_pretrained('bert-base-uncased')
model_fine_tuned = torch.load(fine_tuned_filepath, map_location=torch.device(device_name))

c:\users\krystian kurek\.conda\envs\nlp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly

In [6]:
import tqdm

In [7]:
import os
import glob

In [8]:
models = {'model_pretrained': model_pretrained, 'model_fine_tuned': model_fine_tuned}


In [9]:
concepts = pd.read_csv(concepts_filepath)

In [15]:
for model_name, model in models.items():
    concepts_emb = return_embeddings_for_concepts(concepts['STR'],model)
    concepts_emb['concept_name'] = concepts['STR']
    concepts_emb[['concept_name'] + list(range(len(concepts_emb.columns)-1))]
    filename = embeddings_output_filepath.replace('.csv',f'_{model_name}.csv')
    concepts_emb.to_csv(filename, index=False)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  9.75it/s]
